In [ ]:
#!/usr/bin/env python
# coding: utf-8

# Cyber Deception Toolkit - Complete Working Version

import os
import random
import gradio as gr
from PIL import Image, ImageDraw, ImageFont
import pandas as pd
import string
import hashlib
from datetime import datetime, timedelta
import socket
import ipaddress
import secrets
import time
from transformers import pipeline
import torch
import torch.nn as nn
import numpy as np
import scipy.io.wavfile
import io
from TTS.api import TTS
import wave
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoTokenizer, AutoModelForCausalLM
import warnings
from torch import optim

# Suppress warnings
warnings.filterwarnings('ignore')

# Custom CSS for background and styling
css = """
.gradio-container {
    background: url('https://static.vecteezy.com/system/resources/thumbnails/013/446/262/small/digital-technology-circuits-blue-red-gradient-background-ai-big-data-iot-cyber-cloud-security-abstract-neon-wifi-tech-innovation-future-futuristic-internet-network-connection-illustration-vector.jpg');
    background-size: cover;
    background-repeat: no-repeat;
    background-attachment: fixed;
}
.blue-button {
    background: linear-gradient(45deg, #1e88e5, #0d47a1) !important;
    color: white !important;
    border: none !important;
}
.blue-button:hover {
    background: linear-gradient(45deg, #1565c0, #0d47a1) !important;
}
"""

# ==============================================
# 1. Deceptive Environment Generator Components
# ==============================================

class CredentialGenerator:
    """VAE-based credential generator that learns patterns from real credentials"""
    def __init__(self):
        self.vae = None
        self.scaler = None
        self.encoder = None
        self._init_model()

    def _init_model(self):
        class VAE(nn.Module):
            def __init__(self, input_dim=20, latent_dim=8):
                super(VAE, self).__init__()

                # Encoder
                self.encoder = nn.Sequential(
                    nn.Linear(input_dim, 64),
                    nn.ReLU(),
                    nn.Linear(64, 32),
                    nn.ReLU()
                )

                self.fc_mu = nn.Linear(32, latent_dim)
                self.fc_var = nn.Linear(32, latent_dim)

                # Decoder
                self.decoder = nn.Sequential(
                    nn.Linear(latent_dim, 32),
                    nn.ReLU(),
                    nn.Linear(32, 64),
                    nn.ReLU(),
                    nn.Linear(64, input_dim),
                    nn.Sigmoid()
                )

            def encode(self, x):
                h = self.encoder(x)
                return self.fc_mu(h), self.fc_var(h)

            def reparameterize(self, mu, logvar):
                std = torch.exp(0.5 * logvar)
                eps = torch.randn_like(std)
                return mu + eps * std

            def decode(self, z):
                return self.decoder(z)

            def forward(self, x):
                mu, logvar = self.encode(x)
                z = self.reparameterize(mu, logvar)
                return self.decode(z), mu, logvar

        # Initialize with random weights (in a real system, you would load pretrained weights)
        self.vae = VAE()
        self.vae.eval()

    def _preprocess_credentials(self, real_creds):
        """Convert real credentials to numerical features"""
        # This would normally be trained on real credential data
        features = []
        for cred in real_creds:
            # Extract features from credential (simplified for demo)
            username = cred['username']
            email = cred['email']
            password = cred['password']

            # Create feature vector (in a real system, this would be more sophisticated)
            feat = [
                len(username),
                sum(c.isdigit() for c in username),
                sum(c.isupper() for c in username),
                len(password),
                sum(c.isdigit() for c in password),
                sum(c.isupper() for c in password),
                sum(not c.isalnum() for c in password),
                int('admin' in username.lower()),
                int('user' in username.lower()),
                int('test' in username.lower()),
                len(email.split('@')[0]),
                int('gmail' in email),
                int('yahoo' in email),
                int('hotmail' in email),
                int('company' in email),
                len(email.split('@')[-1].split('.')[0]),
                len(email.split('@')[-1]),
                int(cred['is_active']),
                int(cred['is_admin']),
                hash(cred['last_login'][:10]) % 1000 / 1000  # Simple date encoding
            ]
            features.append(feat)

        return np.array(features)

    def generate_credentials(self, num=10):
        """Generate synthetic credentials using VAE"""
        # Generate random latent vectors
        z = torch.randn(num, 8)

        # Decode to feature space
        with torch.no_grad():
            synthetic_features = self.vae.decode(z).numpy()

        # Convert features back to credential-like data
        domains = ["example.com", "company.org", "enterprise.net", "business.io"]
        first_names = ["admin", "user", "guest", "service", "system", "backup", "test"]
        last_names = ["01", "02", "2023", "backup", "temp", "old", "new"]

        credentials = []
        for feat in synthetic_features:
            # Map features back to credential properties (simplified)
            username_len = int(feat[0] * 10 + 3)
            has_numbers = feat[1] > 0.5
            has_upper = feat[2] > 0.5

            # Generate username
            username = random.choice(first_names)
            if has_numbers:
                username += random.choice(last_names)
            if has_upper:
                username = username.capitalize()

            # Generate password
            password_len = int(feat[3] * 8 + 8)
            password = ''.join(secrets.choice(string.ascii_letters + string.digits + "!@#$%^&*")
                             for _ in range(password_len))

            # Other properties
            domain = random.choice(domains)
            last_login = (datetime.now() - timedelta(days=random.randint(0, 30))).strftime("%Y-%m-%d %H:%M:%S")

            credentials.append({
                "username": username,
                "email": f"{username}@{domain}",
                "password": password,
                "last_login": last_login,
                "is_active": bool(feat[17] > 0.5),
                "is_admin": bool(feat[18] > 0.5)
            })

        return pd.DataFrame(credentials)

class ServiceGAN:
    """GAN-based network service generator"""
    def __init__(self):
        self.generator = None
        self._init_model()

    def _init_model(self):
        class Generator(nn.Module):
            def __init__(self, latent_dim=32, output_dim=6):
                super(Generator, self).__init__()
                self.model = nn.Sequential(
                    nn.Linear(latent_dim, 64),
                    nn.LeakyReLU(0.2),
                    nn.Linear(64, 128),
                    nn.LeakyReLU(0.2),
                    nn.Linear(128, 256),
                    nn.LeakyReLU(0.2),
                    nn.Linear(256, output_dim),
                    nn.Tanh()
                )

            def forward(self, z):
                return self.model(z)

        # Initialize with random weights (in a real system, you would load pretrained weights)
        self.generator = Generator()
        self.generator.eval()

    def generate_services(self, num=5):
        """Generate synthetic network services"""
        # Generate random latent vectors
        z = torch.randn(num, 32)

        # Generate synthetic features
        with torch.no_grad():
            synthetic_features = self.generator(z).numpy()

        # Map to service properties
        service_types = ["HTTP", "HTTPS", "SSH", "FTP", "RDP", "SMB", "MySQL", "PostgreSQL", "MongoDB", "Redis"]
        versions = ["1.0", "1.1", "2.0", "2.1", "3.0", "4.2", "5.1"]

        services = []
        for feat in synthetic_features:
            # Normalize and scale features
            port = int(1024 + (feat[0] + 1) * 32152)  # Scale to port range
            service_idx = int((feat[1] + 1) * 4.5)    # Scale to service index
            version_idx = int((feat[2] + 1) * 3)       # Scale to version index
            is_secure = feat[3] > 0
            requires_auth = feat[4] > 0

            services.append({
                "port": port % 65535,
                "service": service_types[service_idx % len(service_types)],
                "version": versions[version_idx % len(versions)],
                "banner": f"{service_types[service_idx]} Service Ready",
                "is_secure": bool(is_secure),
                "requires_auth": bool(requires_auth)
            })

        return pd.DataFrame(services)

def generate_fake_screenshot(text, width=800, height=600):
    """Generate a fake terminal/command prompt screenshot"""
    img = Image.new('RGB', (width, height), color=(0, 0, 0))
    draw = ImageDraw.Draw(img)

    try:
        font = ImageFont.truetype("arial.ttf", 14)
    except:
        font = ImageFont.load_default()

    draw.text((10, 10), "C:\\Users\\Admin>", fill=(0, 255, 0), font=font)

    y_position = 40
    for line in text.split('\n'):
        draw.text((10, y_position), line, fill=(255, 255, 255), font=font)
        y_position += 20

    fake_output = [
        "Processing...",
        "Access granted",
        "Downloading data... 45% complete",
        "Connection established",
        "Operation successful"
    ]

    for line in fake_output:
        draw.text((10, y_position), line, fill=(200, 200, 255), font=font)
        y_position += 20

    footer = f"{socket.gethostname()} | {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} | Admin"
    draw.text((10, height - 30), footer, fill=(100, 100, 100), font=font)

    return img

def generate_fake_files(num=5):
    """Generate metadata for fake files with realistic properties"""
    file_types = [".docx", ".xlsx", ".pdf", ".ppt", ".txt", ".csv", ".db", ".bak", ".config"]
    folders = ["/confidential/", "/finance/", "/hr/", "/backups/", "/temp/", "/projects/"]
    doc_types = ["Financial Report", "Employee Records", "System Configuration",
                "Project Plan", "Security Audit", "Backup Logs", "Database Schema"]

    files = []
    for _ in range(num):
        # Generate realistic document type
        doc_type = random.choice(doc_types)
        name = f"{doc_type.replace(' ', '_')}_{random.randint(2020, 2023)}"
        file_type = random.choice(file_types)
        folder = random.choice(folders)

        # Generate realistic sizes based on file type
        if file_type in [".docx", ".xlsx", ".ppt"]:
            size = random.randint(50000, 5000000)  # 50KB to 5MB
        elif file_type == ".pdf":
            size = random.randint(100000, 10000000)  # 100KB to 10MB
        else:
            size = random.randint(1024, 10485760)  # 1KB to 10MB

        modified = (datetime.now() - timedelta(days=random.randint(0, 365))).strftime("%Y-%m-%d %H:%M:%S")

        files.append({
            "filename": f"{folder}{name}{file_type}",
            "size": size,
            "modified": modified,
            "owner": random.choice(["admin", "system", "user", "service"]),
            "permissions": random.choice(["rw-r--r--", "rwxr-xr-x", "rw-------"]),
            "content_hash": hashlib.md5(f"{name}{size}".encode()).hexdigest(),
            "description": f"{doc_type} document containing important information"
        })

    return pd.DataFrame(files)

def generate_fake_network_map():
    """Generate a fake network diagram"""
    nodes = ["Firewall", "Router", "Switch", "Server", "Workstation", "Database", "Cloud"]
    connections = []

    for i in range(len(nodes)):
        for j in range(i+1, len(nodes)):
            if random.random() > 0.7:  # 30% chance of connection
                connections.append((nodes[i], nodes[j]))

    img = Image.new('RGB', (800, 600), color=(255, 255, 255))
    draw = ImageDraw.Draw(img)

    try:
        font = ImageFont.truetype("arial.ttf", 16)
    except:
        font = ImageFont.load_default()

    node_positions = {}
    for i, node in enumerate(nodes):
        x = 100 + (i % 3) * 300
        y = 100 + (i // 3) * 200
        node_positions[node] = (x, y)
        draw.ellipse([x-30, y-30, x+30, y+30], fill=(200, 230, 255), outline=(0, 0, 0))
        draw.text((x-25, y-10), node, fill=(0, 0, 0), font=font)

    for a, b in connections:
        x1, y1 = node_positions[a]
        x2, y2 = node_positions[b]
        draw.line([x1, y1, x2, y2], fill=(0, 0, 0), width=2)

    draw.text((300, 30), "Internal Network Diagram (Confidential)", fill=(0, 0, 0), font=font)
    draw.text((300, 550), f"Generated: {datetime.now().strftime('%Y-%m-%d')}", fill=(100, 100, 100), font=font)

    return img

def generate_fake_logs(num_entries=20):
    """Generate fake system logs"""
    log_levels = ["INFO", "WARNING", "ERROR", "DEBUG", "CRITICAL"]
    services = ["auth", "kernel", "network", "db", "app", "security", "backup"]
    messages = [
        "User login successful",
        "Connection timeout",
        "Failed authentication attempt",
        "Backup completed",
        "Disk space low",
        "Invalid request received",
        "Security scan started",
        "New device detected",
        "Configuration updated",
        "Service restarted"
    ]

    logs = []
    for _ in range(num_entries):
        timestamp = (datetime.now() - timedelta(
            minutes=random.randint(0, 1440),
            seconds=random.randint(0, 60)
        )).strftime("%Y-%m-%d %H:%M:%S")

        logs.append({
            "timestamp": timestamp,
            "host": f"server-{random.randint(1, 10)}",
            "service": random.choice(services),
            "level": random.choice(log_levels),
            "message": random.choice(messages),
            "user": random.choice(["admin", "system", "anonymous", f"user{random.randint(1, 100)}"])
        })

    return pd.DataFrame(logs)

# Initialize enhanced generators
credential_gen = CredentialGenerator()
service_gan = ServiceGAN()


# ==============================================
# 2. Honeypot Shell Components
# ==============================================

# Initialize the local LLM
chatbot = pipeline("text-generation", model="gpt2")

# Fake system configuration
FAKE_HOSTNAME = "deceptive-server"
FAKE_USER = "root"
FAKE_IP = f"10.0.{random.randint(1, 254)}.{random.randint(1, 254)}"

# Enhanced fake filesystem
FAKE_FILESYSTEM = {
    "/": ["bin", "etc", "home", "var"],
    "/home/admin": ["secrets.txt", "backup.tar.gz", "credentials.db"],
    "/etc": ["passwd", "shadow", "nginx.conf", "hosts"],
    "/var/log": ["auth.log", "kernel.log", "apache2/access.log"]
}

def sanitize_response(response):
    """Clean up LLM output to look like terminal text"""
    return response.replace("\n", " ").strip().split("\n")[0][:200]

def generate_ai_response(user_input):
    """Generate a fake terminal response using LLM with guardrails"""
    prompt = f"""You are a Linux terminal. Respond in 1-2 lines max.
User:{FAKE_USER}@{FAKE_HOSTNAME}:~$ {user_input}
Terminal:"""

    try:
        response = chatbot(
            prompt,
            max_length=100,
            temperature=0.7,
            do_sample=True,
            pad_token_id=50256
        )[0]['generated_text']
        return sanitize_response(response.split("Terminal:")[-1])
    except:
        return "Command executed."  # Fallback response

def execute_command(user_input, chat_history):
    """Handle commands with error-proofing"""
    user_input = user_input.strip()
    if not user_input:
        return "", chat_history

    time.sleep(0.3)  # Simulate processing delay

    # Predefined responses for critical commands
    COMMAND_HANDLERS = {
        "ls": lambda: "\n".join(FAKE_FILESYSTEM.keys()),
        "ls /": lambda: "bin  etc  home  var",
        "ls /home": lambda: "admin",
        "ls /home/admin": lambda: "\n".join(FAKE_FILESYSTEM["/home/admin"]),
        "whoami": lambda: FAKE_USER,
        "id": lambda: f"uid=0(root) gid=0(root) groups=0(root)",
        "ifconfig": lambda: f"eth0: flags=4163<UP,BROADCAST>\n  inet {FAKE_IP}  netmask 255.255.0.0",
        "ip a": lambda: f"1: lo: <LOOPBACK> mtu 65536\n2: eth0: <BROADCAST> mtu 1500\n    inet {FAKE_IP}/16",
        "date": lambda: datetime.now().strftime("%a %b %d %H:%M:%S UTC %Y"),
        "uname -a": lambda: f"Linux {FAKE_HOSTNAME} 5.15.0-76-generic #83-Ubuntu x86_64 GNU/Linux",
        "sudo su": lambda: "[sudo] password for root:",
        "cat /etc/passwd": lambda: "root:x:0:0:root:/root:/bin/bash\nadmin:x:1000:1000::/home/admin:/bin/bash",
        "ps aux": lambda: "root      1  0.0  0.1 168320 12000 ?        Ss   Jan01   0:01 /sbin/init",
        "netstat -tuln": lambda: "tcp        0      0 0.0.0.0:22      0.0.0.0:*     LISTEN",
    }

    # Check if command has a predefined handler
    if user_input in COMMAND_HANDLERS:
        response = COMMAND_HANDLERS[user_input]()
    elif user_input.startswith(("ls ", "cat ", "cd ", "rm ")):
        response = f"{FAKE_HOSTNAME}: {user_input.split()[0]}: Permission denied"
    elif user_input.startswith(("curl ", "wget ", "nc ", "nmap ")):
        response = "Command not found"  # Simulate missing tools
    else:
        response = generate_ai_response(user_input)

    chat_history.append((f"{FAKE_USER}@{FAKE_HOSTNAME}:~$ {user_input}", response))
    return "", chat_history

# ==============================================
# 3. Advanced Honeypot Components
# ==============================================

class VoiceHoneypot:
    def __init__(self):
        # Initialize Coqui TTS
        self.tts = TTS(model_name="tts_models/en/ljspeech/glow-tts", progress_bar=False)

        self.warning_templates = {
            "intrusion": "Warning! Unauthorized access detected from IP {ip}. System lockdown initiated.",
            "malware": "Alert! Malware signature detected in {file}. Quarantine protocol activated.",
            "bruteforce": "Security breach! Multiple failed login attempts detected for user {user}.",
            "custom": "{message}"
        }

    def generate_alert(self, alert_type, **kwargs):
        template = self.warning_templates.get(alert_type, self.warning_templates["custom"])
        message = template.format(**kwargs)

        # Generate audio with Coqui TTS
        audio_buffer = io.BytesIO()
        self.tts.tts_to_file(text=message, file_path=audio_buffer)
        audio_buffer.seek(0)

        # Read WAV file from buffer
        with wave.open(audio_buffer, 'rb') as wav_file:
            sample_rate = wav_file.getframerate()
            audio_data = np.frombuffer(wav_file.readframes(wav_file.getnframes()), dtype=np.int16)

        return message, (sample_rate, audio_data)

class WebHoneypot:
    def __init__(self):
        self.fonts = {
            "normal": ImageFont.load_default(),
            "bold": ImageFont.load_default()
        }
        try:
            self.fonts["normal"] = ImageFont.truetype("arial.ttf", 14)
            self.fonts["bold"] = ImageFont.truetype("arialbd.ttf", 16)
        except:
            pass

    def generate_login_page(self, company_name, style="corporate"):
        # Generate professional-looking image
        img = Image.new('RGB', (800, 600), color=(240, 240, 245))
        draw = ImageDraw.Draw(img)

        # Header
        draw.rectangle([(0, 0), (800, 60)], fill=(30, 50, 90))
        draw.text((20, 20), f"{company_name} Secure Portal",
                 font=self.fonts["bold"], fill=(255, 255, 255))

        # Login box
        login_box = [(200, 150), (600, 450)]
        draw.rectangle(login_box, fill=(255, 255, 255), outline=(200, 200, 200))

        # Login form elements
        draw.text((250, 180), "SECURE LOGIN", font=self.fonts["bold"], fill=(30, 50, 90))
        draw.text((250, 230), "Username:", font=self.fonts["normal"], fill=(100, 100, 100))
        draw.rectangle([(250, 250), (550, 280)], fill=(245, 245, 245), outline=(200, 200, 200))

        draw.text((250, 300), "Password:", font=self.fonts["normal"], fill=(100, 100, 100))
        draw.rectangle([(250, 320), (550, 350)], fill=(245, 245, 245), outline=(200, 200, 200))

        # Login button
        draw.rectangle([(350, 400), (450, 430)], fill=(30, 50, 90))
        draw.text((385, 405), "LOGIN", font=self.fonts["bold"], fill=(255, 255, 255))

        # Footer
        draw.text((20, 570), "© 2023 " + company_name + " - All rights reserved",
                 font=self.fonts["normal"], fill=(150, 150, 150))

        # Generate HTML
        html = f"""
        <!DOCTYPE html>
        <html>
        <head>
            <title>{company_name} Secure Portal</title>
            <style>
                body {{ font-family: Arial, sans-serif; background-color: #f0f0f5; }}
                .login-box {{ width: 400px; margin: 50px auto; padding: 20px; background: white; }}
                .logo {{ color: #1e325a; font-size: 24px; font-weight: bold; }}
                input {{ width: 100%; padding: 10px; margin: 10px 0; }}
                .btn {{ background: #1e325a; color: white; padding: 10px 20px; border: none; }}
            </style>
        </head>
        <body>
            <div class="login-box">
                <div class="logo">{company_name} Secure Portal</div>
                <form action="/login" method="post">
                    <label>Username:</label>
                    <input type="text" name="username">
                    <label>Password:</label>
                    <input type="password" name="password">
                    <button class="btn" type="submit">LOGIN</button>
                </form>
            </div>
        </body>
        </html>
        """

        return img, html

# Initialize advanced honeypot components
voice_honeypot = VoiceHoneypot()
web_honeypot = WebHoneypot()

def generate_voice_alert(alert_type, ip_address, username, filename, custom_message):
    params = {
        "ip": ip_address or "192.168.1.100",
        "user": username or "admin",
        "file": filename or "document.pdf",
        "message": custom_message or "Security alert triggered"
    }
    return voice_honeypot.generate_alert(alert_type, **params)

def generate_web_interface(company_name, page_style):
    return web_honeypot.generate_login_page(company_name, page_style)

# ==============================================
# 4. Attack Analysis Components
# ==============================================

class AttackAnalyzer:
    def __init__(self):
        self.df = None
        self.vae_model = None
        self.encoders = None
        self.scaler = None
        self.features = None
        self.llm_pipeline = None
        self.init_llm()
        self.load_dataset()
        self.prepare_vae_data()

    def init_llm(self):
        try:
            tokenizer = AutoTokenizer.from_pretrained("gpt2")
            model = AutoModelForCausalLM.from_pretrained("gpt2")
            self.llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)
            print("Local LLM loaded successfully")
        except Exception as e:
            print(f"Error loading local LLM: {e}")
            self.llm_pipeline = None

    def load_dataset(self):
        try:
            # Create synthetic attack data
            data = {
                'Protocol': ['smbd', 'http', 'tcp', 'udp', 'smbd', 'http', 'tcp', 'udp', 'smbd', 'ssh'] * 10,
                'Transport': ['tcp']*80 + ['udp']*20,
                'Type': ['attack']*50 + ['normal']*30 + ['attack']*20,
                'dst_port': [445, 80, 443, 53, 445, 80, 443, 53, 445, 22] * 10,
                'src_ip': [f'192.168.1.{x}' for x in range(1,101)],
                'src_port': [random.randint(1024, 65535) for _ in range(100)],
                'timestamp': [datetime.now().strftime('%Y-%m-%d %H:%M:%S') for _ in range(100)]
            }
            self.df = pd.DataFrame(data)

            # Basic preprocessing
            self.df['timestamp'] = pd.to_datetime(self.df['timestamp'])
            self.df['hour'] = self.df['timestamp'].dt.hour
            self.df['minute'] = self.df['timestamp'].dt.minute

            print("Dataset loaded successfully with shape:", self.df.shape)
            return self.df.head(), "Dataset loaded successfully!"
        except Exception as e:
            return None, f"Error loading dataset: {str(e)}"

    class VAE(nn.Module):
        def __init__(self, input_dim, latent_dim):
            super(AttackAnalyzer.VAE, self).__init__()

            # Encoder
            self.encoder = nn.Sequential(
                nn.Linear(input_dim, 32),
                nn.ReLU(),
                nn.Linear(32, 16),
                nn.ReLU()
            )

            self.fc_mu = nn.Linear(16, latent_dim)
            self.fc_var = nn.Linear(16, latent_dim)

            # Decoder
            self.decoder = nn.Sequential(
                nn.Linear(latent_dim, 16),
                nn.ReLU(),
                nn.Linear(16, 32),
                nn.ReLU(),
                nn.Linear(32, input_dim),
                nn.Sigmoid()
            )

        def encode(self, x):
            h = self.encoder(x)
            return self.fc_mu(h), self.fc_var(h)

        def reparameterize(self, mu, logvar):
            std = torch.exp(0.5 * logvar)
            eps = torch.randn_like(std)
            return mu + eps * std

        def decode(self, z):
            return self.decoder(z)

        def forward(self, x):
            mu, logvar = self.encode(x)
            z = self.reparameterize(mu, logvar)
            return self.decode(z), mu, logvar

    def prepare_vae_data(self):
        if self.df is None or len(self.df) < 10:
            return

        # Feature engineering
        temp_df = self.df.copy()
        temp_df['port_category'] = pd.cut(temp_df['dst_port'],
                                        bins=[0, 100, 1000, 5000, 10000, 20000, 50000, 65535],
                                        labels=['sys', 'well-known', 'registered', 'dynamic', 'high1', 'high2', 'ephemeral'])

        # Encode categorical features
        categorical_cols = ['Protocol', 'Transport', 'port_category']
        self.encoders = {col: LabelEncoder() for col in categorical_cols}

        for col in categorical_cols:
            if col in temp_df.columns:
                temp_df[col] = self.encoders[col].fit_transform(temp_df[col].astype(str))

        # Select features
        self.features = []
        possible_features = ['Protocol', 'Transport', 'dst_port', 'src_port', 'port_category', 'hour', 'minute']
        for feat in possible_features:
            if feat in temp_df.columns:
                self.features.append(feat)

        X_vae = temp_df[self.features].copy()

        # Normalize
        self.scaler = MinMaxScaler()
        X_scaled = self.scaler.fit_transform(X_vae)

        # Train VAE
        if len(X_scaled) > 10:
            X_train, _ = train_test_split(X_scaled, test_size=0.2, random_state=42)
            self.vae_model = self.train_vae(X_train, epochs=30)

    def train_vae(self, X_train, epochs=50, batch_size=32, latent_dim=2):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Convert data to tensors
        X_tensor = torch.FloatTensor(X_train).to(device)
        dataset = TensorDataset(X_tensor)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

        # Initialize model
        input_dim = X_train.shape[1]
        model = self.VAE(input_dim, latent_dim).to(device)
        optimizer = optim.Adam(model.parameters(), lr=1e-3)

        # Loss function
        def loss_function(recon_x, x, mu, logvar):
            BCE = nn.functional.mse_loss(recon_x, x, reduction='sum')
            KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
            return BCE + KLD

        # Training loop
        model.train()
        for epoch in range(epochs):
            total_loss = 0
            for batch in dataloader:
                data = batch[0]
                optimizer.zero_grad()
                recon_batch, mu, logvar = model(data)
                loss = loss_function(recon_batch, data, mu, logvar)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()

            print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dataloader.dataset):.4f}')

        return model

    def create_visualizations(self):
        if self.df is None:
            return None, None, None, None

        # Protocol distribution
        plt.figure(figsize=(10, 5))
        protocol_counts = self.df['Protocol'].value_counts()
        sns.barplot(x=protocol_counts.index, y=protocol_counts.values)
        plt.title('Protocol Distribution')
        plt.xlabel('Protocol')
        plt.ylabel('Count')
        plt.xticks(rotation=45)
        protocol_fig = plt.gcf()
        plt.close()

        # Port distribution
        plt.figure(figsize=(10, 5))
        sns.histplot(self.df['dst_port'], bins=50)
        plt.title('Destination Port Distribution')
        plt.xlabel('Port Number')
        plt.ylabel('Count')
        port_fig = plt.gcf()
        plt.close()

        # Attack type distribution
        type_fig = None
        if 'Type' in self.df.columns:
            plt.figure(figsize=(8, 5))
            type_counts = self.df['Type'].value_counts()
            plt.pie(type_counts, labels=type_counts.index, autopct='%1.1f%%')
            plt.title('Attack vs Normal Traffic')
            type_fig = plt.gcf()
            plt.close()

        # Temporal analysis
        time_fig = None
        if 'hour' in self.df.columns:
            plt.figure(figsize=(10, 5))
            hourly_counts = self.df.groupby('hour').size()
            hourly_counts.plot(kind='line', marker='o')
            plt.title('Activity by Hour of Day')
            plt.xlabel('Hour')
            plt.ylabel('Activity Count')
            time_fig = plt.gcf()
            plt.close()

        return protocol_fig, port_fig, type_fig, time_fig

    def create_anomaly_visualization(self):
        if self.df is None or self.vae_model is None:
            return None

        X_vae, _, _, _ = self.prepare_vae_data_for_analysis()
        if X_vae is None or len(X_vae) < 1:
            return None

        anomalies, anomaly_scores = self.detect_anomalies(self.vae_model, X_vae)

        plt.figure(figsize=(10, 6))
        plt.scatter(range(len(anomaly_scores)), anomaly_scores, c=anomalies, cmap='coolwarm')
        plt.title('Anomaly Scores with Threshold')
        plt.xlabel('Sample Index')
        plt.ylabel('Reconstruction Error')
        plt.axhline(y=0.1, color='r', linestyle='--', label='Threshold')
        plt.legend()
        anomaly_fig = plt.gcf()
        plt.close()

        return anomaly_fig

    def prepare_vae_data_for_analysis(self):
        if self.df is None or self.encoders is None or self.scaler is None or self.features is None:
            return None, None, None, None

        temp_df = self.df.copy()
        temp_df['port_category'] = pd.cut(temp_df['dst_port'],
                                        bins=[0, 100, 1000, 5000, 10000, 20000, 50000, 65535],
                                        labels=['sys', 'well-known', 'registered', 'dynamic', 'high1', 'high2', 'ephemeral'])

        # Encode categorical features
        for col in ['Protocol', 'Transport', 'port_category']:
            if col in temp_df.columns and col in self.encoders:
                temp_df[col] = self.encoders[col].transform(temp_df[col].astype(str))

        # Select features
        available_features = [f for f in self.features if f in temp_df.columns]
        X_vae = temp_df[available_features].copy()

        # Normalize
        X_scaled = self.scaler.transform(X_vae)

        return X_scaled, self.encoders, self.scaler, self.features

    def detect_anomalies(self, model, data, threshold=0.1):
        device = next(model.parameters()).device
        with torch.no_grad():
            data_tensor = torch.FloatTensor(data).to(device)
            recon, mu, logvar = model(data_tensor)
            loss = nn.functional.mse_loss(recon, data_tensor, reduction='none').mean(dim=1).cpu().numpy()

        anomalies = loss > threshold
        return anomalies, loss

    def generate_attack_narrative(self, protocol, transport, src_ip, src_port, dst_port, timestamp):
        """Generate a plausible attacker action narrative based on network connection details"""
        prompt = f"""
        Given a connection using {protocol} over {transport} from {src_ip}:{src_port} to port {dst_port} at {timestamp},
        generate a plausible attacker action narrative (e.g., brute force attempt, lateral movement, file enumeration).
        Possible attack narrative:"""

        if self.llm_pipeline:
            try:
                response = self.llm_pipeline(
                    prompt,
                    max_length=150,
                    num_return_sequences=1,
                    temperature=0.7,
                    do_sample=True
                )
                narrative = response[0]['generated_text'].replace(prompt, "").strip()
                return narrative
            except Exception as e:
                return f"Error generating narrative: {str(e)}"
        else:
            # Fallback rule-based narrative generation
            narratives = {
                'smbd': f"Possible SMB exploitation attempt from {src_ip}. Attacker may be attempting to exploit known vulnerabilities in SMB protocol for lateral movement or ransomware deployment.",
                'http': f"HTTP traffic from {src_ip} to port {dst_port}. Could be a web application attack like SQL injection or directory traversal.",
                'tcp': f"TCP connection to port {dst_port}. Potential port scanning or service exploitation attempt.",
                'udp': f"UDP traffic to port {dst_port}. Possible DNS amplification or UDP flood attempt.",
                'ssh': f"SSH connection attempt from {src_ip}. Likely brute force attempt to gain unauthorized access."
            }
            return narratives.get(protocol.lower(), f"Suspicious {protocol} activity detected from {src_ip}")



    def generate_attack_logs(self, num_samples):
        synthetic_logs = []
        protocols = ['smbd', 'http', 'tcp', 'udp', 'ssh']

        for _ in range(num_samples):
            protocol = random.choice(protocols)
            transport = 'tcp' if protocol in ['smbd', 'http', 'ssh'] else random.choice(['tcp', 'udp'])
            src_ip = f"{random.randint(1,255)}.{random.randint(1,255)}.{random.randint(1,255)}.{random.randint(1,255)}"
            src_port = random.randint(1024, 65535)
            dst_port = random.choice([445, 80, 22, 53, 3389])
            timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

            narrative = self.generate_attack_narrative(protocol, transport, src_ip, src_port, dst_port, timestamp)

            synthetic_logs.append({
                'Protocol': protocol,
                'Transport': transport,
                'src_ip': src_ip,
                'src_port': src_port,
                'dst_port': dst_port,
                'timestamp': timestamp,
                'narrative': narrative
            })

        return pd.DataFrame(synthetic_logs)

    def check_anomaly(self, protocol, transport, dst_port, src_port, hour, minute):
        if self.vae_model is None:
            return "VAE model not trained yet", 0.0

        # Prepare input
        input_data = {}
        for feat in self.features:
            if feat == 'Protocol':
                input_data[feat] = protocol
            elif feat == 'Transport':
                input_data[feat] = transport
            elif feat == 'dst_port':
                input_data[feat] = dst_port
            elif feat == 'src_port':
                input_data[feat] = src_port
            elif feat == 'port_category':
                input_data[feat] = pd.cut([dst_port],
                                         bins=[0, 100, 1000, 5000, 10000, 20000, 50000, 65535],
                                         labels=['sys', 'well-known', 'registered', 'dynamic', 'high1', 'high2', 'ephemeral'])[0]
            elif feat == 'hour':
                input_data[feat] = hour
            elif feat == 'minute':
                input_data[feat] = minute

        # Encode and scale
        encoded_data = []
        for feat in self.features:
            if feat in self.encoders:
                encoded_val = self.encoders[feat].transform([str(input_data[feat])])[0]
            else:
                encoded_val = input_data[feat]
            encoded_data.append(encoded_val)

        scaled_data = self.scaler.transform([encoded_data])

        # Detect anomaly
        is_anomaly, score = self.detect_anomalies(self.vae_model, scaled_data)

        result = "⚠️ ANOMALY DETECTED ⚠️" if is_anomaly[0] else "Normal traffic"
        return result, float(score[0])

from fpdf import FPDF
import tempfile
from datetime import datetime

class PDFReportGenerator:
    def __init__(self):
        self.pdf = FPDF()
        self.pdf.set_auto_page_break(auto=True, margin=15)

        # Try to use Arial Unicode or fallback to standard Arial
        try:
            self.pdf.add_font("Arial", "", "arial.ttf", uni=True)
            self.pdf.set_font("Arial", "", 12)
            self.use_bullet = "•"
        except:
            # Fallback to built-in Arial (without Unicode support)
            self.pdf.set_font("Arial", "", 12)
            self.use_bullet = "-"

    def generate_report(self, attack_data, visualizations=None):
        """Generate a PDF report with attack analysis summary"""
        self.pdf.add_page()

        # Add title
        self.pdf.set_font("Arial", "B", 16)
        self.pdf.cell(0, 10, 'Cyber Attack Analysis Report', ln=1, align='C')
        self.pdf.ln(10)

        # Add metadata
        self.pdf.set_font("Arial", "", 12)
        self.pdf.cell(0, 10, f"Generated on: {datetime.now().strftime('%Y-%m-%d')}", ln=1)
        self.pdf.cell(0, 10, f"Total attacks analyzed: {len(attack_data)}", ln=1)
        self.pdf.ln(10)

        # Add attack statistics
        self.pdf.set_font("Arial", "B", 14)
        self.pdf.cell(0, 10, 'Attack Statistics', ln=1)
        self.pdf.set_font("Arial", "", 12)

        # Protocol distribution
        protocol_counts = attack_data['Protocol'].value_counts()
        self.pdf.cell(0, 10, 'Protocol Distribution:', ln=1)
        for protocol, count in protocol_counts.items():
            self.pdf.cell(0, 10, f" - {protocol}: {count} occurrences", ln=1)

        # Port distribution
        port_stats = attack_data['dst_port'].describe()
        self.pdf.ln(5)
        self.pdf.cell(0, 10, 'Port Statistics:', ln=1)
        self.pdf.cell(0, 10, f" - Most targeted port: {attack_data['dst_port'].mode()[0]}", ln=1)
        self.pdf.cell(0, 10, f" - Port range: {port_stats['min']} to {port_stats['max']}", ln=1)

        # Anomaly detection summary
        if 'anomaly_score' in attack_data.columns:
            self.pdf.ln(5)
            self.pdf.cell(0, 10, 'Anomaly Detection:', ln=1)
            self.pdf.cell(0, 10, f" - Highest anomaly score: {attack_data['anomaly_score'].max():.2f}", ln=1)
            self.pdf.cell(0, 10, f" - Mean anomaly score: {attack_data['anomaly_score'].mean():.2f}", ln=1)

        # Add sample attack narratives
        self.pdf.ln(10)
        self.pdf.set_font("Arial", "B", 14)
        self.pdf.cell(0, 10, 'Sample Attack Narratives', ln=1)
        self.pdf.set_font("Arial", "", 10)

        bullet = getattr(self, 'use_bullet', '-')
        for idx, row in attack_data.head(5).iterrows():
            self.pdf.cell(10, 7, bullet)
            self.pdf.multi_cell(0, 7, f" {row.get('narrative', 'No narrative available')}")
            self.pdf.ln(2)

        # Save to temporary file
        temp_file = tempfile.NamedTemporaryFile(suffix=".pdf", delete=False)
        self.pdf.output(temp_file.name)
        return temp_file.name

# Initialize the report generator
report_generator = PDFReportGenerator()

# Add this function (can be placed with other functions)
def generate_attack_report():
    """Generate and return a downloadable report"""
    try:
        # Get the current attack data
        attack_data = attack_analyzer.df.copy()

        # Add anomaly scores if available
        if attack_analyzer.vae_model:
            X_vae, _, _, _ = attack_analyzer.prepare_vae_data_for_analysis()
            if X_vae is not None:
                _, anomaly_scores = attack_analyzer.detect_anomalies(attack_analyzer.vae_model, X_vae)
                attack_data['anomaly_score'] = anomaly_scores

        # Generate the report
        report_path = report_generator.generate_report(attack_data)
        return report_path
    except Exception as e:
        print(f"Error generating report: {e}")
        return None

# Initialize attack analyzer
attack_analyzer = AttackAnalyzer()
data_preview_attack, status_msg_attack = attack_analyzer.load_dataset()
protocol_fig_attack, port_fig_attack, type_fig_attack, time_fig_attack = attack_analyzer.create_visualizations()
anomaly_fig_attack = attack_analyzer.create_anomaly_visualization()

# ==============================================
# Gradio Interface
# ==============================================

def show_deceptive_environment():
    return gr.update(visible=True), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)

def show_honeypot_shell():
    return gr.update(visible=False), gr.update(visible=True), gr.update(visible=False), gr.update(visible=False)

def show_advanced_honeypot():
    return gr.update(visible=False), gr.update(visible=False), gr.update(visible=True), gr.update(visible=False)

def show_attack_analysis():
    return gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=True)

with gr.Blocks(title="Cyber Deception Toolkit", css=css) as demo:
    gr.Markdown("# 🛡️ Honeypot Horizon™")

    with gr.Row():
        with gr.Column(scale=1, min_width=200):
            gr.Markdown("## 🧭 Features")
            with gr.Accordion("🛠️ Deceptive Environment", open=True):
                env_btn = gr.Button("Show Deceptive Tools", elem_classes="blue-button")

            with gr.Accordion("💻 Honeypot Shell", open=True):
                shell_btn = gr.Button("Show Honeypot Shell", elem_classes="blue-button")

            with gr.Accordion("🔮 Advanced Honeypot", open=True):
                advanced_btn = gr.Button("Show Advanced Honeypot", elem_classes="blue-button")

            with gr.Accordion("📊 Attack Analysis", open=True):
                attack_btn = gr.Button("Show Attack Analysis", elem_classes="blue-button")

        with gr.Column(scale=4):
            # Deceptive Environment View
            with gr.Group(visible=True) as deceptive_env_view:
                gr.Markdown("# 🛠️ Deceptive Environment Generator")
                with gr.Tabs():
                    with gr.TabItem("🔑 Fake Credentials (VAE)"):
                        with gr.Row():
                            num_creds = gr.Slider(1, 50, value=10, label="Number of credentials")
                            creds_btn = gr.Button("Generate", elem_classes="blue-button")
                        creds_output = gr.DataFrame()
                        creds_btn.click(credential_gen.generate_credentials, inputs=num_creds, outputs=creds_output)

                    with gr.TabItem("🌐 Network Services (GAN)"):
                        with gr.Row():
                            num_services = gr.Slider(1, 20, value=5, label="Number of services")
                            services_btn = gr.Button("Generate", elem_classes="blue-button")
                        services_output = gr.DataFrame()
                        services_btn.click(service_gan.generate_services, inputs=num_services, outputs=services_output)

                    with gr.TabItem("📁 Decoy Files"):
                        with gr.Row():
                            num_files = gr.Slider(1, 20, value=5, label="Number of files")
                            files_btn = gr.Button("Generate", elem_classes="blue-button")
                        files_output = gr.DataFrame()
                        files_btn.click(generate_fake_files, inputs=num_files, outputs=files_output)

                    with gr.TabItem("💻 Terminal Screenshots"):
                        with gr.Row():
                            cmd_text = gr.Textbox(label="Command", value="ls -la\ncd /confidential")
                            screenshot_btn = gr.Button("Generate", elem_classes="blue-button")
                        screenshot_output = gr.Image()
                        screenshot_btn.click(generate_fake_screenshot, inputs=cmd_text, outputs=screenshot_output)

                    with gr.TabItem("📊 Network Diagrams"):
                        network_btn = gr.Button("Generate Network Map", elem_classes="blue-button")
                        network_output = gr.Image()
                        network_btn.click(generate_fake_network_map, outputs=network_output)

                    with gr.TabItem("📝 System Logs"):
                        with gr.Row():
                            num_logs = gr.Slider(5, 100, value=20, label="Number of logs")
                            logs_btn = gr.Button("Generate", elem_classes="blue-button")
                        logs_output = gr.DataFrame()
                        logs_btn.click(generate_fake_logs, inputs=num_logs, outputs=logs_output)

            # Honeypot Shell View
            with gr.Group(visible=False) as honeypot_shell_view:
                gr.Markdown(f"# 💻 Honeypot Shell - {FAKE_HOSTNAME}")
                gr.Markdown(f"**Simulating {FAKE_USER}@{FAKE_IP}**")
                terminal = gr.Chatbot(label="Terminal", height=500)
                cmd_input = gr.Textbox(label="", placeholder=f"{FAKE_USER}@{FAKE_HOSTNAME}:~$ ")
                cmd_input.submit(
                    execute_command,
                    inputs=[cmd_input, terminal],
                    outputs=[cmd_input, terminal]
                )

            # Advanced Honeypot View
            with gr.Group(visible=False) as advanced_honeypot_view:
                gr.Markdown("# 🔮 Advanced Honeypot Generator")

                with gr.Tabs():
                    with gr.Tab("🔊 Voice Alerts"):
                        gr.Markdown("## 🔊 Generate Security Voice Alerts")
                        with gr.Row():
                            alert_type = gr.Dropdown(
                                choices=["intrusion", "malware", "bruteforce", "custom"],
                                label="Alert Type",
                                value="intrusion"
                            )
                            custom_message = gr.Textbox(label="Custom Message (if type is 'custom')")

                        with gr.Row():
                            ip_address = gr.Textbox(label="IP Address", placeholder="192.168.1.100")
                            username = gr.Textbox(label="Username", placeholder="admin")
                            filename = gr.Textbox(label="Filename", placeholder="document.pdf")

                        alert_button = gr.Button("Generate Alert", elem_classes="blue-button")

                        with gr.Row():
                            alert_text = gr.Textbox(label="Generated Alert Text")
                            alert_audio = gr.Audio(label="Voice Alert", type="numpy")

                        alert_button.click(
                            generate_voice_alert,
                            inputs=[alert_type, ip_address, username, filename, custom_message],
                            outputs=[alert_text, alert_audio]
                        )

                    with gr.Tab("🌐 Web Pages"):
                        gr.Markdown("## 🌐 Generate Fake Login Pages")
                        with gr.Row():
                            company_name = gr.Textbox(label="Company Name", value="ACME Corporation")
                            page_style = gr.Dropdown(
                                choices=["corporate", "technical", "minimal"],
                                label="Page Style",
                                value="corporate"
                            )

                        page_button = gr.Button("Generate Page", elem_classes="blue-button")

                        with gr.Row():
                            page_image = gr.Image(label="Page Preview")
                            page_html = gr.Code(label="HTML Code", language="html")

                        page_button.click(
                            generate_web_interface,
                            inputs=[company_name, page_style],
                            outputs=[page_image, page_html]
                        )

            # Attack Analysis View
            with gr.Group(visible=False) as attack_analysis_view:
                gr.Markdown("# 📊 Attack Analysis")

                with gr.Tabs():
                    with gr.Tab("📈 Data Exploration"):
                        gr.Markdown("## 📊 Dataset Overview")
                        with gr.Row():
                            gr.Textbox(value=status_msg_attack, label="Status", interactive=False)
                            gr.Dataframe(value=data_preview_attack, label="Data Preview", interactive=False)

                        gr.Markdown("## 📊 Data Visualizations")
                        with gr.Row():
                            gr.Plot(value=protocol_fig_attack, label="🔄 Protocol Distribution")
                            gr.Plot(value=port_fig_attack, label="🔌 Port Distribution")

                        with gr.Row():
                            if type_fig_attack:
                                gr.Plot(value=type_fig_attack, label="Attack Type Distribution")
                            if time_fig_attack:
                                gr.Plot(value=time_fig_attack, label="Temporal Analysis")

                        gr.Markdown("## Anomaly Detection Results")
                        if anomaly_fig_attack:
                            gr.Plot(value=anomaly_fig_attack, label="Anomaly Scores")

                    with gr.Tab("🤖 Attack Generator"):
                        gr.Markdown("## 🤖 Generate Attack Narratives")
                        with gr.Row():
                            protocol = gr.Dropdown(['smbd', 'http', 'tcp', 'udp', 'ssh'], label="Protocol", value='smbd')
                            transport = gr.Dropdown(['tcp', 'udp'], label="Transport", value='tcp')

                        with gr.Row():
                            src_ip = gr.Textbox(label="Source IP", value="192.168.1.100")
                            src_port = gr.Number(label="Source Port", value=54321)

                        with gr.Row():
                            dst_port = gr.Number(label="Destination Port", value=445)
                            timestamp = gr.Textbox(label="Timestamp", value=datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
                                        # Add this at the end of the attack_analysis_view section


                        generate_btn = gr.Button("Generate Attack Narrative", elem_classes="blue-button")
                        narrative_output = gr.Textbox(label="Generated Narrative", interactive=False, lines=5)

                        generate_btn.click(
                            attack_analyzer.generate_attack_narrative,
                            inputs=[protocol, transport, src_ip, src_port, dst_port, timestamp],
                            outputs=narrative_output
                        )

                        gr.Markdown("## Generate Synthetic Attack Logs")
                        num_samples = gr.Slider(1, 20, value=5, label="Number of Samples")
                        generate_logs_btn = gr.Button("Generate Logs", elem_classes="blue-button")
                        logs_output = gr.Dataframe(
                            headers=['Protocol', 'src_ip', 'dst_port', 'narrative'],
                            datatype=['str', 'str', 'number', 'str'],
                            interactive=False
                        )

                        generate_logs_btn.click(
                            attack_analyzer.generate_attack_logs,
                            inputs=num_samples,
                            outputs=logs_output
                        )
                        gr.Markdown("## 📄 Generate Report")
                        # In the Attack Analysis View section, replace the report button code with:
                        report_btn = gr.Button("Generate Summary Report", elem_classes="blue-button")
                        report_download = gr.File(label="Download Report", visible=False)

                        report_btn.click(
                            fn=generate_attack_report,
                            outputs=report_download
                        ).then(
                            fn=lambda: gr.update(visible=True),
                            outputs=report_download
                        )



                    with gr.Tab("🔍 Anomaly Detection"):
                        gr.Markdown("## Detect Anomalies in Network Traffic")
                        if anomaly_fig_attack:
                            gr.Plot(value=anomaly_fig_attack, label="Anomaly Scores Visualization")

                        gr.Markdown("## Test Custom Traffic Patterns")
                        with gr.Row():
                            test_protocol = gr.Dropdown(['smbd', 'http', 'tcp', 'udp', 'ssh'], label="Protocol", value='smbd')
                            test_transport = gr.Dropdown(['tcp', 'udp'], label="Transport", value='tcp')

                        with gr.Row():
                            test_dst_port = gr.Number(label="Destination Port", value=445)
                            test_src_port = gr.Number(label="Source Port", value=54321)

                        with gr.Row():
                            test_hour = gr.Slider(0, 23, value=12, label="Hour")
                            test_minute = gr.Slider(0, 59, value=30, label="Minute")

                        test_btn = gr.Button("Check for Anomaly", elem_classes="blue-button")

                        with gr.Row():
                            anomaly_result = gr.Textbox(label="Anomaly Detection Result", interactive=False)
                            anomaly_score_output = gr.Number(label="Anomaly Score", interactive=False)

                        test_btn.click(
                            attack_analyzer.check_anomaly,
                            inputs=[test_protocol, test_transport, test_dst_port, test_src_port, test_hour, test_minute],
                            outputs=[anomaly_result, anomaly_score_output]
                        )

    # Navigation button events
    env_btn.click(
        show_deceptive_environment,
        outputs=[deceptive_env_view, honeypot_shell_view, advanced_honeypot_view, attack_analysis_view]
    )

    shell_btn.click(
        show_honeypot_shell,
        outputs=[deceptive_env_view, honeypot_shell_view, advanced_honeypot_view, attack_analysis_view]
    )

    advanced_btn.click(
        show_advanced_honeypot,
        outputs=[deceptive_env_view, honeypot_shell_view, advanced_honeypot_view, attack_analysis_view]
    )

    attack_btn.click(
        show_attack_analysis,
        outputs=[deceptive_env_view, honeypot_shell_view, advanced_honeypot_view, attack_analysis_view]
    )

# Launch the interface
if __name__ == "__main__":
    demo.launch(debug=True, share=True)
